In [1]:
import pickle
import os 
import nltk
import demjson3
from nltk.corpus import wordnet
import re

In [2]:
def are_synonyms_split(word_list,word2):
    """第一个是一个列表"""
    right=[]

    for i in word_list:
        for j in word2:
            right.append(are_synonyms(i,j))

    if True in right:
        return True 
    else:
        return False

def has_hypernym_or_hyponym_split(word_list,word2):
    """第一个是一个列表"""
    right=[]

    for i in word_list:
        for j in word2:
            right.append(has_hypernym_or_hyponym(i,j))

    if True in right:
        return True 
    else:
        return False

def are_synonyms(word1, word2):
    # 获取word1的所有同义词集
    synsets1 = wordnet.synsets(word1)
    
    # 获取word2的所有同义词集
    synsets2 = wordnet.synsets(word2)
  
    # 检查是否有交集
    for synset1 in synsets1:
        for synset2 in synsets2:
            if synset1 == synset2:
                return True
    
    # 计算路径相似度作为备选方案
    for synset1 in synsets1:
        for synset2 in synsets2:
            if synset1.path_similarity(synset2) is not None and synset1.path_similarity(synset2) > 0.8:
                return True
    
    return False

def has_hypernym_or_hyponym(word1, word2):
    # 获取word1的所有同义词集
    synsets1 = wordnet.synsets(word1)
    
    # 获取word2的所有同义词集
    synsets2 = wordnet.synsets(word2)
    
    # 检查是否存在上位词或下位词关系
    for synset1 in synsets1:
        for synset2 in synsets2:
            # 检查synset1是否是synset2的上位词
            if synset1 in synset2.hypernyms():
                return True
            
            # 检查synset2是否是synset1的上位词
            if synset2 in synset1.hypernyms():
                return True
    
    return False

def count_common_elements_with_repetition(list1, list2):
    # 创建字典来存储每个元素在两个列表中的出现次数
    count_dict1 = {}
    count_dict2 = {}
    
    # 统计第一个列表中每个元素的出现次数
    for element in list1:
        if element in count_dict1:
            count_dict1[element] += 1
        else:
            count_dict1[element] = 1
    
    # 统计第二个列表中每个元素的出现次数
    for element in list2:
        if element in count_dict2:
            count_dict2[element] += 1
        else:
            count_dict2[element] = 1
    
    # 计算共同元素的数量
    common_count = 0
    for element in count_dict1:
        if element in count_dict2:
            # 取两个列表中该元素出现次数的较小值
            common_count += min(count_dict1[element], count_dict2[element])
    
    return common_count

In [5]:
def compute_right_single(ground_truth,predict):
    """
        计算单张图片的预测准确率
        分成四个 计数，位置，关系，total
    """

    category_right=0
    number_right=0
    loc_right=0

    ready_list=[]

    for i in predict['number'].keys():
        for j in ground_truth['number'].keys():

            if j in ready_list:
                continue

            i_split=re.split(r'[_-]',i)
            j_split=re.split(r'[_-]',j)


            if are_synonyms_split(i_split,j_split):        ##标签对应上了
                ready_list.append(j)        #加入已经完成配对

                category_right+=1
                if predict['number'][i]==ground_truth['number'][j]:     #统计数量一样，加1
                    number_right+=1


                ###计算位置对了几个
                loc_right+=count_common_elements_with_repetition(list(predict['location'][i].values()),ground_truth['location'][j])
                break


    rel_right=0        ###计算关系预测对了几个   逻辑上 关系三元组，一个个去比

    for i in predict['relationship']:
        for j in ground_truth['relationship']:

            i_split=re.split(r'[\s_-]',i[0])
            j_split=re.split(r'[\s_-]',j[0])        

            if are_synonyms_split(i_split,j_split):        

                i_split=re.split(r'[\s_-]',i[1])
                j_split=re.split(r'[\s_-]',j[1]) 

                if len(i_split)<=2:
                    i_split=i_split[0]
                else:
                    i_split=i_split[1]

                if len(j_split)<=2:
                    j_split=j_split[0]
                else:
                    j_split=j_split[1]

                if are_synonyms(i_split,j_split):        

                    i_split=re.split(r'[\s_-]',i[2])
                    j_split=re.split(r'[\s_-]',j[2])

                    
                    if are_synonyms_split(i_split,j_split):        
                        rel_right+=1
                        break

    ### 计算总数
    
    number_num=len(ground_truth['number'].keys())
    loc_num=sum(ground_truth['number'].values())
    rel_num=len(ground_truth['relationship'])

    gen_num=len(predict['number'].keys())
    gen_loc=sum(predict['number'].values())
    gen_rel=len(predict['relationship'])

    ###计算 precision
    if gen_num==0:
        number_precision=0
        category_precision=0
    else:
        category_precision=category_right/gen_num
        number_precision=number_right/gen_num
    if gen_loc==0:
        loc_precision=0
    else:
        loc_precision=loc_right/gen_loc
    if gen_rel==0:
        rel_precision=0
    else:
        rel_precision=rel_right/gen_rel

    ### 计算recall
    if number_num==0:
        number_recall=1
        category_recall=1
    else:
        category_recall=category_right/number_num
        number_recall=number_right/number_num

    if loc_num==0:
        loc_recall=1
    else:
        loc_recall=loc_right/loc_num
    
    if rel_num==0:
        rel_recall=1
    else:
        rel_recall=rel_right/rel_num

    #### 计算F1
    if category_precision==0 and category_recall==0:
        category_f1=0
    else:
        category_f1=2*(category_precision*category_recall)/(category_precision+category_recall)
    if number_precision==0 and number_recall==0:
        num_f1=0
    else:
        num_f1=2*(number_precision*number_recall)/(number_precision+number_recall)
    
    if loc_precision==0 and loc_recall==0:
        loc_f1=0
    else:
        loc_f1=2*(loc_precision*loc_recall)/(loc_precision+loc_recall)
    
    if rel_precision==0 and rel_recall==0:
        rel_f1=0
    else:
        rel_f1=2*(rel_precision*rel_recall)/(rel_precision+rel_recall)

    return category_precision,number_precision,loc_precision,rel_precision,category_recall,number_recall,loc_recall,rel_recall,category_f1,num_f1,loc_f1,rel_f1


In [28]:
with open('ground_truth.pickle','rb') as f:
    ground_truth_total=pickle.load(f)

In [29]:
acc_list=[]

answer_list=os.listdir('gemini_answer')

for i,answer in enumerate(answer_list):

    with open(os.path.join('gemini_answer',answer),'rb') as f:
        data=pickle.load(f)
        print(answer)
    predict=demjson3.decode(data)
    ground_truth=ground_truth_total[i+299]

    acc_list.append(list(compute_right_single(ground_truth,predict)))



    

446.pickle
447.pickle
451.pickle
452.pickle
454.pickle
455.pickle
457.pickle
458.pickle
460.pickle
461.pickle
462.pickle
463.pickle
464.pickle
465.pickle
467.pickle
470.pickle
472.pickle
473.pickle
474.pickle
475.pickle
476.pickle
478.pickle
479.pickle
480.pickle
484.pickle
485.pickle
488.pickle
489.pickle
491.pickle
493.pickle
494.pickle
497.pickle
498.pickle
499.pickle
500.pickle
501.pickle
502.pickle
507.pickle
508.pickle
509.pickle
510.pickle
512.pickle
513.pickle
514.pickle
515.pickle
517.pickle
518.pickle
521.pickle
524.pickle
525.pickle
526.pickle
529.pickle
530.pickle
531.pickle
532.pickle
533.pickle
535.pickle
536.pickle
537.pickle
538.pickle
540.pickle
543.pickle
544.pickle
545.pickle
549.pickle
551.pickle
554.pickle
556.pickle
558.pickle
559.pickle
560.pickle
563.pickle
564.pickle
565.pickle
566.pickle
567.pickle
568.pickle
569.pickle
570.pickle
572.pickle
573.pickle
574.pickle
576.pickle
579.pickle
580.pickle
582.pickle
583.pickle
584.pickle
585.pickle
586.pickle
587.pickle

In [30]:
import numpy as np

acc_numpy=np.array(acc_list)

In [31]:
np.set_printoptions(precision=4,suppress=True)
acc_numpy.mean(axis=0)

array([0.6477, 0.3526, 0.2217, 0.0357, 0.1615, 0.0908, 0.0555, 0.0095,
       0.2488, 0.1389, 0.08  , 0.0142])

In [34]:
acc_numpy.max()

1.0

In [32]:
(acc_numpy>0.5)[:,4:8].mean(1).mean()

0.0

In [26]:
with open('ground_truth.pickle','rb') as f:
    ground_truth_total=pickle.load(f)

In [ ]:
acc_list=[]

answer_list=os.listdir('gpt_answer')

for i,answer in enumerate(answer_list):

    with open(os.path.join('gpt_answer',answer),'rb') as f:
        data=pickle.load(f)
        print(answer)
    predict=demjson3.decode(data)
    ground_truth=ground_truth_total[i+299]

    acc_list.append(list(compute_right_single(ground_truth,predict)))



In [22]:
import numpy as np

acc_numpy=np.array(acc_list)

acc_numpy.mean(axis=0)

array([0.4924, 0.2685, 0.1725, 0.0051, 0.0908, 0.0584, 0.0324, 0.0001,
       0.1472, 0.0921, 0.0504, 0.0002])

In [24]:
(acc_numpy>0.5)[:,4:8].mean(1).mean()

0.0

In [4]:
with open('vg_data/VG_ground_truth.pickle','rb') as f:
    ground_truth_total=pickle.load(f)

In [13]:
acc_list=[]
k=0

answer_list=os.listdir('VG_gemini_answer')

for i,answer in enumerate(answer_list[::-1]):

    with open(os.path.join('VG_gemini_answer',answer),'rb') as f:
        data=pickle.load(f)
        print(answer)

    try:
        predict=demjson3.decode(data)
    except demjson3.JSONDecodeError:
        continue

    
    
    ground_truth=ground_truth_total[int(answer[:-7])]

    if len(ground_truth['relationship'])==0:
        continue

    acc_list.append(list(compute_right_single(ground_truth,predict)))

    k+=1
    if k==100:
        break



2339172.pickle
2339171.pickle
2339170.pickle
2339168.pickle
2339167.pickle
2339166.pickle
2339165.pickle
2339164.pickle
2339163.pickle
2339162.pickle
2339161.pickle
2339160.pickle
2339159.pickle
2339158.pickle
2339157.pickle
2339156.pickle
2339155.pickle
2339154.pickle
2339153.pickle
2339152.pickle
2339151.pickle
2339150.pickle
2339149.pickle
2339148.pickle
2339147.pickle
2339146.pickle
2339145.pickle
2339144.pickle
2339143.pickle
2339142.pickle
2339141.pickle
2339140.pickle
2339139.pickle
2339138.pickle
2339137.pickle
2339136.pickle
2339135.pickle
2339134.pickle
2339133.pickle
2339132.pickle
2339131.pickle
2339130.pickle
2339129.pickle
2339127.pickle
2339126.pickle
2339125.pickle
2339124.pickle
2339123.pickle
2339122.pickle
2339075.pickle
2339074.pickle
2339073.pickle
2339072.pickle
2339071.pickle
2339070.pickle
2339069.pickle
2339068.pickle
2339067.pickle
2339066.pickle
2339065.pickle
2339064.pickle
2339063.pickle
2339062.pickle
2339061.pickle
2339060.pickle
2339059.pickle
2339058.pi

In [65]:
predict

{'number': {'donut': '153'},
 'location': {'donut': {'1': 'top left corner',
   '2': 'center',
   '3': 'top right corner',
   '4': 'left',
   '5': 'center',
   '6': 'right',
   '7': 'left lower corner',
   '8': 'lower',
   '9': 'right lower corner'}},
 'relationship': []}

In [15]:
import numpy as np

acc_numpy=np.array(acc_list)
np.set_printoptions(precision=4)
acc_numpy.mean(axis=0)

array([0.5708, 0.3358, 0.1169, 0.0488, 0.2304, 0.1355, 0.063 , 0.0327,
       0.3   , 0.1748, 0.0662, 0.0296])

In [17]:
(acc_numpy>0.55)[:,4:8].mean(1).mean()

0.025

In [18]:
with open('vg_data/VG_ground_truth.pickle','rb') as f:
    ground_truth_total=pickle.load(f)

In [19]:
acc_list=[]

answer_list=os.listdir('VG_gpt_answer')

k=0

for i,answer in enumerate(answer_list[::-1]):

    with open(os.path.join('VG_gpt_answer',answer),'rb') as f:
        data=pickle.load(f)
        print(answer)
    
    try:
        predict=demjson3.decode(data)
    except demjson3.JSONDecodeError:
        continue
    ground_truth=ground_truth_total[int(answer[:-7])]

    if len(ground_truth['relationship'])==0:
        continue

    acc_list.append(list(compute_right_single(ground_truth,predict)))
    k+=1
    if k==100:
        break



2339172.pickle
2339171.pickle
2339170.pickle
2339169.pickle
2339168.pickle
2339167.pickle
2339166.pickle
2339165.pickle
2339164.pickle
2339163.pickle
2339162.pickle
2339161.pickle
2339160.pickle
2339159.pickle
2339158.pickle
2339157.pickle
2339156.pickle
2339155.pickle
2339154.pickle
2339153.pickle
2339152.pickle
2339151.pickle
2339150.pickle
2339149.pickle
2339148.pickle
2339147.pickle
2339146.pickle
2339145.pickle
2339144.pickle
2339143.pickle
2339142.pickle
2339141.pickle
2339140.pickle
2339139.pickle
2339138.pickle
2339137.pickle
2339136.pickle
2339135.pickle
2339134.pickle
2339133.pickle
2339132.pickle
2339131.pickle
2339130.pickle
2339129.pickle
2339127.pickle
2339126.pickle
2339125.pickle
2339124.pickle
2339123.pickle
2339122.pickle
2339075.pickle
2339074.pickle
2339073.pickle
2339072.pickle
2339071.pickle
2339070.pickle
2339069.pickle
2339068.pickle
2339067.pickle
2339066.pickle
2339065.pickle
2339064.pickle
2339063.pickle
2339062.pickle
2339061.pickle
2339060.pickle
2339059.pi

In [20]:
import numpy as np

acc_numpy=np.array(acc_list)

acc_numpy.mean(axis=0)

array([0.6083, 0.3233, 0.144 , 0.0408, 0.2586, 0.1475, 0.0719, 0.0295,
       0.3427, 0.1918, 0.0886, 0.0255])

In [22]:
(acc_numpy>0.55)[:,4:8].mean(1).mean()

0.025

In [ ]:
with open('ground_truth.pickle','rb') as f:
    ground_truth_total=pickle.load(f)

In [ ]:
acc_list=[]

answer_list=os.listdir('Qwen_answer')

for i,answer in enumerate(answer_list):

    with open(os.path.join('Qwen_answer',answer),'rb') as f:
        data=pickle.load(f)
        print(answer)
    predict=demjson3.decode(data)
    ground_truth=ground_truth_total[i+299]

    acc_list.append(list(compute_right_single(ground_truth,predict)))


446.pickle
447.pickle
451.pickle
452.pickle
454.pickle
455.pickle
457.pickle
458.pickle
460.pickle
461.pickle
462.pickle
463.pickle
464.pickle
465.pickle
467.pickle
470.pickle
472.pickle
473.pickle
474.pickle
475.pickle
476.pickle
478.pickle
479.pickle
480.pickle
484.pickle
485.pickle
488.pickle
489.pickle
491.pickle
493.pickle
494.pickle
497.pickle
498.pickle
499.pickle
500.pickle
501.pickle
502.pickle
507.pickle
508.pickle
509.pickle
510.pickle
512.pickle
513.pickle
514.pickle
515.pickle
517.pickle
518.pickle
521.pickle
524.pickle
525.pickle
526.pickle
529.pickle
530.pickle
531.pickle
532.pickle
533.pickle
535.pickle
536.pickle
537.pickle
538.pickle
540.pickle
543.pickle
544.pickle
545.pickle
549.pickle
551.pickle
554.pickle
556.pickle
558.pickle
559.pickle
560.pickle
563.pickle
564.pickle
565.pickle
566.pickle
567.pickle
568.pickle
569.pickle
570.pickle
572.pickle
573.pickle
574.pickle
576.pickle
579.pickle
580.pickle
582.pickle
583.pickle
584.pickle
585.pickle
586.pickle
587.pickle

In [ ]:
import numpy as np

acc_numpy=np.array(acc_list)

In [ ]:
np.set_printoptions(precision=4,suppress=True)
acc_numpy.mean(axis=0)

array([0.4163, 0.2084, 0.1151, 0.016 , 0.0877, 0.0508, 0.0229, 0.0055,
       0.1387, 0.0778, 0.0345, 0.0075])

In [ ]:

(acc_numpy>0.5)[:,4:8].mean(1).mean()

0.005050505050505051

In [ ]:
with open('VG_data/VG_ground_truth.pickle','rb') as f:
    ground_truth_total=pickle.load(f)

In [ ]:
acc_list=[]

k=0

answer_list=os.listdir('VG_Qwen_answer')

for i,answer in enumerate(answer_list[::-1]):

    with open(os.path.join('VG_Qwen_answer',answer),'rb') as f:
        data=pickle.load(f)
        print(answer)

    try:                                    ##跳过错误
        predict=demjson3.decode(data)
    except demjson3.JSONDecodeError:
        continue

    ground_truth=ground_truth_total[int(answer[:-7])]

    if len(ground_truth['relationship'])==0:
        continue

    acc_list.append(list(compute_right_single(ground_truth,predict)))

    k+=1
    if k==100:
        break




2339172.pickle
2339171.pickle
2339170.pickle
2339169.pickle
2339168.pickle
2339167.pickle
2339166.pickle
2339165.pickle
2339164.pickle
2339163.pickle
2339162.pickle
2339161.pickle
2339160.pickle
2339159.pickle
2339158.pickle
2339157.pickle
2339156.pickle
2339155.pickle
2339154.pickle
2339153.pickle
2339152.pickle
2339151.pickle
2339150.pickle
2339149.pickle
2339148.pickle
2339147.pickle
2339146.pickle
2339145.pickle
2339144.pickle
2339143.pickle
2339142.pickle
2339141.pickle
2339140.pickle
2339139.pickle
2339138.pickle
2339137.pickle
2339136.pickle
2339135.pickle
2339134.pickle
2339133.pickle
2339132.pickle
2339131.pickle
2339130.pickle
2339129.pickle
2339127.pickle
2339126.pickle
2339125.pickle
2339124.pickle
2339123.pickle
2339122.pickle
2339121.pickle
2339120.pickle
2339119.pickle
2339118.pickle
2339117.pickle
2339116.pickle
2339115.pickle
2339114.pickle
2339113.pickle
2339112.pickle
2339110.pickle
2339109.pickle
2339108.pickle
2339107.pickle
2339106.pickle
2339105.pickle
2339104.pi

In [ ]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)
acc_numpy=np.array(acc_list)

In [ ]:
acc_numpy.mean(axis=0)

array([0.5417, 0.3593, 0.1285, 0.0537, 0.3017, 0.2032, 0.0764, 0.0505,
       0.3412, 0.2257, 0.0853, 0.0377])

In [ ]:
(acc_numpy>0.55)[:,4:8].mean(1).mean()

0.05